In [ ]:
# Testing Cell

from aviary.api import Aircraft, AviaryProblem
from aviary.utils.doctape import get_variable_name, glue_variable, glue_class_functions

glue_variable(get_variable_name(Aircraft.CrewPayload.Design.NUM_PASSENGERS), md_code=True)
glue_variable(get_variable_name(Aircraft.CrewPayload.NUM_PASSENGERS), md_code=True)
glue_variable(get_variable_name(Aircraft.CrewPayload.NUM_BUSINESS_CLASS), md_code=True)
glue_variable(get_variable_name(Aircraft.CrewPayload.NUM_FIRST_CLASS), md_code=True)
glue_variable(get_variable_name(Aircraft.CrewPayload.NUM_ECONOMY_CLASS), md_code=True)

current_glued_vars = []
glue_class_functions(AviaryProblem, current_glued_vars, prefix='prob', md_code=True)

# Multi-Mission Optimization

It is expected that the users is familiar with methods from level 1 and level 2 before exploring this content.

## Overview

Some of the more advanced capabilities that aviary can perform include a multi-mission optimization. The simplest capability could be described as optimize a single aircraft design considering two or more missions that the aircraft will perform. This is in contrast to the traditional "Design" an aircraft for a single mission, and and then fly the aircraft model in "Off Design" to see how it performs in all other mission scenarios. Multi-mission blurs these lines, allowing the aircraft to be designed by asessing the impact over many different missions simultaneously. As such, an economic mission flown for 70% of missions, a max payload mission flown for 20% of missions, and a max range mission flown for 10% of missions could all simultaneously contribute to the aircraft design. The weightings are specified by the user and ideally would reflect the frequency at which each of the individual missions were flown. At it's most complex, multi-mission could allow the user to optimize multiple different aircraft which have a collection of common parts that are being sized (i.e. engines), and each of those aircraft are also flying multiple different missions. 

The MDAO architecture of multi-mission is uses a single monolithic optimization problem. There is no sub-optimizers in the current implementation. This limits the capabilities for including subsystems, such as uncertainty quantification, that are known to fail with mololithic architectures. Future releases may incorporate sub-optimization as a feature.

In the example provided, a large single aisle passenger aircraft is designed based on a mission with every seat filled, and a second mission where there are no passengers. A weighting is provided by the user to determine how often the missions are full of passengers vs. how often the flights are empty. The objective function is based on combining the fuel-burn values from both missions and the optimizers objective is to minimize the weighted fuel-burn. Other objectives, are possible to sepcify, but a maximum range objective is currently not supported but should be added in the future. Behind the scenes of Aviary, both aircraft are setup completely separately, with their own pre-mission, mission, and post-mission. We will need to link some design values together to ensure the aircraft "mirror" eachother for our example case.

## Workflow

The multi-mission interface is slightly modified from the standard aviary methods to provide a more streamlined method of handling inputs. A number of new methods have been added to `methods_for_level2` to support this. The standard workflow for a multi-mission problem is as follows:

- load aviary_values from a .csv (Future Capability)
- aviary_valuse.set_val
- merge_hirarchies
- add AviaryProblem
- {glue:md}`add_aviary_group`
- {glue:md}`build_model`
- add initial guesses for external subsystems (see test_subsystems_within_a_mission.py for example)
- {glue:md}`promote_inputs`
- {glue:md}`add_design_var_default`
- {glue:md}`add_composite_objective`
- {glue:md}`add_composite_objective_adv`
- {glue:md}`add_driver`
- {glue:md}`add_design_variables`
- {glue:md}`setup`
- {glue:md}`set_val`
- {glue:md}`set_design_range`
- {glue:md}`run_aviary_problem`

The ProblemType must be set to MULTI_MISSION otherwise many of the supporting methods will not be enabled. Currently, the {glue:md}`add_aviary_group` method only supports ingesting an aviary_values object. In the future, the capability to sparately load a .csv file and return an aviary_values object will be provided. All modifications to aviary_values must be made before calling {glue:md}`add_aviary_group` because it calls {glue:md}`load_inputs` and then {glue:md}`check_and_preprocess_inputs`, giving the user no opportunity to modify aviary values after loading. This decision was made because {glue:md}`load_inputs` actually sets some unchangable defaults in the background and the user should have the capability to change those settings before they are fixed. {glue:md}`build_model` is called next. This function could be broken out into it's four constituent methods if finer control is desired (see methods for level 2 for details). After calling {glue:md}`build_model`, initial guesses for external subsystems can then be provided since those subsystems are now loaded. Then {glue:md}`promote_inputs` is used to link key design variables between the different aircraft, ensuring that a single optimizer control is mirrored on both aircraft. {glue:md}`add_design_var_default` is a quick way to sepcify both what the optimizer is allowed to control as well as setting a default value for that control. The initial control value can still be modeifed after setup using {glue:md}`set_val` if desired. {glue:md}`add_composite_objective` and {glue:md}`add_composite_objective_adv` are two new ways that the user can specify a composite or combined objective from multiple aircraft and/or missions. For example, you could set the objective to be a combination of fuel-burn from mission1 and maintaince costs from mission2. Each of the different composite objective functions provides a slightly different way of specifying the objective. Users that know the frequency of each of their missions, which can typically be obtained by looking at the flight history of similarly sized aircraft, will find it easy to use {glue:md}`add_composite_objective_adv` to load in their mission frequencies. We then progress to standard {glue:md}`add_driver` and {glue:md}`add_design_variables` calls, followed by {glue:md}`setup` which now also automatically sets input defaults. At this point we can now use {glue:md}`set_val` to create or change any initial guesses if necessary. {glue:md}`set_design_range` is then used to set the design range for each aircraft to a similar value, ensuring subsystems like avionics are mirrored between the two aircraft. It does this by looking through the phase_info of each model and taking the largest one. Lastly, the problem is run using {glue:md}`run_aviary_problem`. A report folder holding each model individually is created and available for users to inspect.

## Design vs. As-Flown
To support the need to design an aircraft with a certain number of seats, but then possibly fly missions with less passengers, a distinction in the metadata was introduced between {glue:md}`Aircraft.CrewPayload.Design.NUM_PASSENGERS` and {glue:md}`Aircraft.CrewPayload.NUM_PASSENGERS`. The individual passenger classes ({glue:md}`Aircraft.CrewPayload.NUM_FIRST_CLASS`, {glue:md}`Aircraft.CrewPayload.NUM_BUSINESS_CLASS`, {glue:md}`Aircraft.CrewPayload.NUM_ECONOMY_CLASS`) also have these distinctions. The Design values represent how many seats are available in the aircraft. Whereas the non-design values represent an as-flow value of how many passengers are on a particular flight. 

A number of checks exist in {glue:md}`check_and_preprocess_inputs` to help the user in the case that incomplete as-flow or design passenger information is provided.

## Example
An example of a multi-mission as well as Setup, Theory, and Results, is presented in [Multi-Mission Examples](../examples/multi_mission).